In [3]:
# Import dependencies
import time
import numpy as np
import pandas as pd

from datetime import timedelta
from pyod.models.abod import ABOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.knn import KNN

from sklearn.model_selection import train_test_split


In [4]:
 # Set seed for reproducible result
seed = 1234
np.random.seed(seed)

print("Loading saved samples...")
t1 = time.perf_counter()
X_load = np.load('samples_32bands_augment_data_06032020.npy')
print(f"Finished loading samples in {timedelta(seconds=time.perf_counter()-t1)}")

print(X_load.shape)
X_load[0:1]  # Show some data for sense


Loading saved samples...
Finished loading samples in 0:00:00.627467
(52254, 40, 64)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [5]:
Num_samples = 15_000
X_load_ = np.reshape(X_load[:Num_samples], newshape=(-1, 40*64))
print(f"X shape: {X_load_.shape}")

# Divide data: train, test
X_train, X_test = train_test_split(X_load_, shuffle=True, test_size=0.2)

print("Fitting classifier...")
t1 = time.perf_counter()
clf = ABOD(contamination=1e-10)
clf.fit(X_train)
print(f"Finished fitting classifier in {timedelta(seconds=time.perf_counter()-t1)}")


X shape: (15000, 2560)
Fitting classifier...


c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finished fitting classifier in 0:10:50.550849


c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\pyod\models\base.py:365: RuntimeWarning: invalid value encountered in greater
  self.labels_ = (self.decision_scores_ > self.threshold_).astype(


In [6]:
y_test_pred = clf.predict(X_test)
y_test_pred

c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\pyod\models\base.py:168: RuntimeWarning: invalid value encountered in greater
  return (pred_score > self.threshold_).astype('int').ravel()


array([0, 0, 0, ..., 0, 0, 0])

In [7]:
X_new = X_test[100:101]
X_new[0, 0:200] = 1
clf.predict(X_new)

c:\users\se.vi.dmitriev\python-virtual-environments\ml\lib\site-packages\pyod\models\base.py:168: RuntimeWarning: invalid value encountered in greater
  return (pred_score > self.threshold_).astype('int').ravel()


array([0])

In [13]:
# With Autoencoder
Num_samples = 1_000_000
X_load_1 = np.reshape(X_load[:Num_samples], newshape=(-1, 40*64))
#X_load_ = X_load[:Num_samples]
print(f"X shape: {X_load_1.shape}")

# Divide data: train, test
X_train1, X_test1 = train_test_split(X_load_1, shuffle=True, test_size=0.2)

print("Fitting classifier...")
t1 = time.perf_counter()
clf1 = AutoEncoder(epochs=30, output_activation='tanh', contamination=1e-10)
clf1.fit(X_train1)
print(f"Finished fitting classifier in {timedelta(seconds=time.perf_counter()-t1)}")


X shape: (52254, 2560)
Fitting classifier...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 2560)              6556160   
_________________________________________________________________
dropout_25 (Dropout)         (None, 2560)              0         
_________________________________________________________________
dense_30 (Dense)             (None, 2560)              6556160   
_________________________________________________________________
dropout_26 (Dropout)         (None, 2560)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                163904    
_________________________________________________________________
dropout_27 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_32 

In [14]:
y_test_pred1 = clf1.predict(X_test1)
y_test_pred1

array([0, 0, 0, ..., 0, 0, 0])

In [22]:
y_test_pred1[y_test_pred1 > 0]

array([1, 1])

In [20]:
X_new1 = X_test1[100:101]
#X_new1[0, 0:50000] = 1
clf1.predict(X_new1)

array([1])